In [1]:
import pandas as pd 
import sqlite3

# OBJECTIF

**ANALYSER ET NETOYER LES DONNEES POUR APPRENDRE LA DATA SCIENCE.** 

---

#### **I - ANALYSE GLOBALE DE TOUTES LES COLONNES.**

In [2]:
data = pd.read_csv("ressources/vintage_site_tx.csv")
data

,tx id,time,auction,color,shirt size,condition
0,8dbf3cfa48b25f32e300f279bb4163c67d5c60cd,3:36 PM,$17.18,Puce,S,used
1,8ccebfdb1a41830bdcc0e926531f3215986b3a90,9:03 AM,$15.19,Aquamarine,L,never worn
2,ae85a421a91cec8baf16ff1e45fb3e239130b584,7:42 PM,$7.58,Violet,L,used
3,85e44f09a04a79182befcfd55e288a855f633e81,10:28 PM,$9.10,Goldenrod,XS,NaN
4,ff750bfca4910cfa7f538fbe5463457839021680,6:38 AM,$8.22,NaN,2XL,used
...,...,...,...,...,...,...
995,6fa16b110f53367cf15e6e1dcffdcb4c2653f1ab,9:56 AM,$5.79,Violet,2XL,NaN
996,2b7b6cbc2224502c7137262575e97a450b336fc2,11:31 AM,$14.07,Purple,S,Used
997,7f2a5141de8a7f752f8f98722393ffa3b0fb1e7a,6:25 AM,$17.63,Yellow,XS,used
998,c4549687dee4d3061fd3c82433367bea083d4d05,2:22 AM,$17.00,Yellow,L,Used


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tx id       1000 non-null   object
 1   time        1000 non-null   object
 2   auction     1000 non-null   object
 3   color       977 non-null    object
 4   shirt size  1000 non-null   object
 5   condition   847 non-null    object
dtypes: object(6)
memory usage: 47.0+ KB


In [4]:
data.drop(['tx id'], axis=1, inplace=True)

#### **II - TRANSFORMONS LA COLONNE auction  EN FLOAT.**

In [5]:
data["auction"] = data["auction"].str[1:].astype(float)
data

,time,auction,color,shirt size,condition
0,3:36 PM,17.18,Puce,S,used
1,9:03 AM,15.19,Aquamarine,L,never worn
2,7:42 PM,7.58,Violet,L,used
3,10:28 PM,9.10,Goldenrod,XS,NaN
4,6:38 AM,8.22,NaN,2XL,used
...,...,...,...,...,...
995,9:56 AM,5.79,Violet,2XL,NaN
996,11:31 AM,14.07,Purple,S,Used
997,6:25 AM,17.63,Yellow,XS,used
998,2:22 AM,17.00,Yellow,L,Used


In [6]:
data['auction'].dtypes

dtype('float64')

#### **III - TRANSFORMONS LE TYPE LA COLONNE TIME EN DATETIME ET CREONS UNE COLONNE CATEGORIELLE.**
CETTE COLONNE AURA LES CARACTERISTIQUES SUIVANTES
1. nuit 1h -> 6h29
2. matin 6h30 -> 11h59
3. midi 12h -> 13h59
4. apres-midi 14h -> 17h59
5. afterwork 18 -> 20h59
6. soir 21h -> 0h59

In [7]:
data['time'] = pd.to_datetime(data['time'],format='%I:%M %p')
data['time']

0     1900-01-01 15:36:00
1     1900-01-01 09:03:00
2     1900-01-01 19:42:00
3     1900-01-01 22:28:00
4     1900-01-01 06:38:00
              ...        
995   1900-01-01 09:56:00
996   1900-01-01 11:31:00
997   1900-01-01 06:25:00
998   1900-01-01 02:22:00
999   1900-01-01 19:47:00
Name: time, Length: 1000, dtype: datetime64[ns]

In [8]:
print(data['time'].dtypes)

datetime64[ns]


In [9]:
tmp = data['time'].dt.hour * 100 + data['time'].dt.minute
tmp = pd.cut(tmp,
       bins=[0,59,629,1159,1359,1759,2059,2400],
       labels=['soir','nuit', 'matin', 'midi', 'apres-midi', 'afterwork', 'soir2'])

In [10]:
data["Time_cat"] = tmp.str.replace('2',"")
data['time'] = data['time'].dt.time
data

,time,auction,color,shirt size,condition,Time_cat
0,15:36:00,17.18,Puce,S,used,apres-midi
1,09:03:00,15.19,Aquamarine,L,never worn,matin
2,19:42:00,7.58,Violet,L,used,afterwork
3,22:28:00,9.10,Goldenrod,XS,NaN,soir
4,06:38:00,8.22,NaN,2XL,used,matin
...,...,...,...,...,...,...
995,09:56:00,5.79,Violet,2XL,NaN,matin
996,11:31:00,14.07,Purple,S,Used,matin
997,06:25:00,17.63,Yellow,XS,used,nuit
998,02:22:00,17.00,Yellow,L,Used,nuit


#### **IV - IDENTIFIONS ET SUPPRIMONS LES DOUBLONS.**

In [11]:
data[data.duplicated()]
# ce dataset ne contient pas de doublons

,time,auction,color,shirt size,condition,Time_cat


#### **V - ANALYSE DES VALEURS MANQUANTES.**

In [12]:
#l'objectif est d'afficher le pourcentage par colonne de valeurs manquantes
(data.isna().sum()/len(data)*100).sort_values(ascending = False)

condition     15.3
color          2.3
time           0.0
auction        0.0
shirt size     0.0
Time_cat       0.0
dtype: float64

#### **VI - EXPORTONS LE FICHIER EN CSV et SQL.**

In [13]:
#EFFECTUONS UN TRI SUR LES DIFFERENTES EDITIONS
Clean_vintage = data[['color','shirt size','auction','condition','time','Time_cat']].sort_values('auction')
Clean_vintage

,color,shirt size,auction,condition,time,Time_cat
88,Purple,2XL,2.02,used,17:52:00,apres-midi
509,Khaki,XL,2.04,NaN,09:47:00,matin
553,Pink,3XL,2.05,used,10:34:00,matin
879,Fuscia,XS,2.06,NaN,00:49:00,soir
263,Pink,2XL,2.07,NaN,23:12:00,soir
...,...,...,...,...,...,...
236,Pink,M,18.92,used,21:09:00,soir
90,Indigo,XL,18.96,Never Worn,01:03:00,nuit
823,Violet,3XL,18.98,Used,22:51:00,soir
818,Turquoise,XL,18.99,used,17:26:00,apres-midi


In [14]:
Clean_vintage.to_csv("ressources/Clean_academy_award.csv", index=False)
con = sqlite3.connect("ressources/Clean_academy_award.db")
Clean_vintage.to_sql("Clean_academy_award",con , index=False, if_exists = "replace")

1000